In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import csv

In [ ]:

head_to_head_links = '/Users/joathcarrera/Desktop/CSE115A/Soccer-Match-Predictor/H2H_Stats/Urls/urls.csv'
output_directory = '/Users/joathcarrera/Desktop/CSE115A/Soccer-Match-Predictor/H2H_Stats/Matchdays/'
api_key = 'b3349e3c5ce9f2853b2b8cee2c0052a7'
api_url = 'https://api.scraperapi.com/'


In [ ]:


# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

with open(head_to_head_links, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row
    
    matchday_count = 1
    matchday_dir = os.path.join(output_directory, f'Matchday_{matchday_count}')
    os.makedirs(matchday_dir, exist_ok=True)
    
    for idx, row in enumerate(reader):
        url = row[0]
        payload = {'api_key': api_key, 'url': url}
        response = requests.get(api_url, params=payload)
        
        if response.status_code != 200:
            print(f"Failed to retrieve URL: {url}")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        scorebox = soup.find('div', class_='scorebox')

        if scorebox:
            teams_data = []
            for div in scorebox.find_all('div', recursive=False):
                team_data = [item.get_text(strip=True) for item in div.find_all('div')]
                teams_data.append(team_data)

            if teams_data:
                # Determine the maximum number of columns
                max_columns = max(len(team_data) for team_data in teams_data)

                # Fill missing values with empty strings
                for team_data in teams_data:
                    while len(team_data) < max_columns:
                        team_data.append('')

                # Generate column names dynamically
                columns = ['Team'] + [f'Column{i}' for i in range(1, max_columns)]

                # Organize the data into a DataFrame
                df = pd.DataFrame(teams_data, columns=columns)

                # Extract a meaningful filename from the URL
                filename = os.path.basename(url).replace('History', '').replace('.html', '') + '.csv'
                filepath = os.path.join(matchday_dir, filename)

                # Save the DataFrame to a CSV file
                df.to_csv(filepath, index=False)

                print(f"CSV file saved to: {filepath}")
            else:
                print(f"No team data found for URL: {url}")
        else:
            print(f"No scorebox found for URL: {url}")

        # Update the matchday directory after every 9 files
        if (idx + 1) % 9 == 0:
            matchday_count += 1
            matchday_dir = os.path.join(output_directory, f'Matchday_{matchday_count}')
            os.makedirs(matchday_dir, exist_ok=True)